<a href="https://colab.research.google.com/github/tuli-pen/simg-llms/blob/master/ArquitecturaRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arquitectura RNN

Este notebook presenta el código necesario para programar una Red Neuronal Recurrente (RNN) y nos permite analizar la arquitectura de este modelo.

### RNN usando Pytorch

In [ ]:
# Importaciones necesarias

import torch
import torch.nn as nn # Módulo de redes neuronales de PyTorch
                      # (NN es la abreviatura de Neural Network)

In [ ]:
# Definimos la arquitectura del modelo

class mi_RNN(nn.Module):  # mi_RNN es una subclase de nn.Module

    # Definimos el constructor de la clase mi_RNN
    def __init__(self, input_size, hidden_size, output_size):

        super(mi_RNN, self).__init__() # Llama al constructor de la clase padre (nn.Module)

        self.hidden_size = hidden_size # Creamos un atributo con el parámetro hidden_size que recibimos

        # Definimos dos capas:
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True) # *1
        self.fc = nn.Linear(hidden_size, output_size) # *2

    # Definimos cómo se pasa la entrada a través de las capas del modelo
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size) # Se inicializa el estado oculto h0 con ceros
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

***1:** En esta línea estamos definiendo una **capa tipo nn.RNN** que toma una entrada de tamaño *input_size* y produce una salida de tamaño *hidden_size*.

*batch_first=True* especifica que el primer parámetro de los datos de entrada será el tamaño del lote (en lugar de la longitud de la secuencia, lo predefinido en Pytorch)



***2:** En esta línea estamos definiendo una **capa fully connected (del tipo nn.Linear)** que toma la salida de la capa RNN (de tamaño *hidden_size*) y la transforma en una salida de tamaño *output_size*.

In [ ]:
# Instanciamos nuestro modelo

input_size = 1
hidden_size = 32
output_size = 1
model = mi_RNN(input_size, hidden_size, output_size)

In [ ]:
# Definimos la función de pérdida y el optimizador

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Creamos datos de ejemplo para el entrenamiento

from torch.utils.data import DataLoader, TensorDataset

train_data = torch.tensor([[[0.1], [0.2], [0.3], [0.4], [0.5]],
                           [[0.6], [0.7], [0.8], [0.9], [1.0]],
                           [[1.1], [1.2], [1.3], [1.4], [1.5]],
                           [[1.6], [1.7], [1.8], [1.9], [2.0]],
                           [[2.1], [2.2], [2.3], [2.4], [2.5]],
                           [[0.8], [0.7], [0.5], [0.9], [0.9]]])  # Datos de entrenamiento
train_labels = torch.tensor([0, 1, 0, 1, 0, 1])  # Etiquetas binarias para los datos de entrenamiento

# DataLoader
train_dataset = TensorDataset(train_data, train_labels)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

In [ ]:
# Entrenamos el modelo

num_epochs = 10
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad() # Se reinician los gradientes
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward() # Retropropagación
        optimizer.step() # Actualiza los pesos

In [ ]:
# Creamos datos de ejemplo para la evaluación

test_data = torch.tensor([[[2.6], [2.7], [2.8], [2.9], [3.0]],
                          [[3.1], [3.2], [3.3], [3.4], [3.5]],
                          [[0.1], [0.4], [2.0], [0.9], [3.0]],
                          [[3.6], [0.2], [3.3], [2.0], [0.5]]])  # Datos de prueba
test_labels = torch.tensor([1, 1, 0, 1])  # Etiquetas binarias para los datos de prueba

# DataLoader
test_dataset = TensorDataset(test_data, test_labels)
test_loader = DataLoader(test_dataset, batch_size=2)

In [ ]:
# Evaluamos el modelo

with torch.no_grad(): # Desactivamos el cálculo de gradientes durante la evaluación
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = torch.round(torch.sigmoid(outputs)) # Convierte los outputs en predicciones binarias
        total += labels.size(0)
        correct += (predicted == labels).sum().item() # *3
accuracy = 100 * correct / total
print("Accuracy:", accuracy, "%")

Accuracy: 50.0 %


***3:** En esta línea estamos calculando el número de predicciones correctas y las estamos agregando al contador "correct".

- **(predicted == labels)** compara las etiquetas predichas (predicted) con las etiquetas reales (labels). Esta operación devuelve un tensor de booleanos. (True indica que la predicción fue correcta y False indica que fue incorrecta).

- **.sum()** suma los elementos del tensor booleano. Como True se interpreta como 1 y False se interpreta como 0 entonces obtenemos el número total de predicciones correctas.

- **.item()** Convierte el resultado de la suma en un número de Python. (Esto es necesario porque .sum() devuelve un tensor de PyTorch).

In [ ]:
# Creamos datos de ejemplo para hacer predicciones

x_new = torch.tensor([[[3.6], [3.7], [3.8], [3.9], [4.0]],
                      [[4.1], [4.2], [4.3], [4.4], [4.5]]])  # Datos de entrada para predicciones

In [ ]:
# Hacemos predicciones usando el modelo

with torch.no_grad():
    predictions = model(x_new)
    predictions = torch.round(torch.sigmoid(predictions))

predictions

tensor([[0.],
        [0.]])